In [2]:
import torch

In [4]:
# Create inputs
inputs = torch.nn.Embedding( 4, 8)

In [6]:
inputs = inputs.weight
inputs = inputs.data

In [ ]:
# Create a batch of inputs. This determines d_in and context_length.
batches = torch.stack((inputs, inputs), dim = 0)


In [3]:
# Create a batch of inputs. This determines d_in and context_length.



In [ ]:
# causal attention mechanism
